<a href="https://colab.research.google.com/github/imbalzy/RecipeQA-FInal-Project-2470/blob/main/preprocess_recipeQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!cp "/content/gdrive/My Drive/Kaggle/kaggle.json" .

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!kaggle datasets download -d jeromeblanchet/recipeqa-nlp-dataset
!unzip recipeqa-nlp-dataset.zip -d data/ > /dev/null

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

100% 2.54G/2.55G [00:41<00:00, 45.0MB/s]
100% 2.55G/2.55G [00:41<00:00, 66.4MB/s]
--2020-11-30 22:38:32--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-30 22:38:33--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-30 22:38:38--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 20

In [2]:
import json
import numpy as np
import re
import tensorflow as tf

def read_file(file_name):
  with open(file_name) as f:
    data = json.load(f)['data']
  textual_cloze = [item for item in data if item["task"]=="textual_cloze"]
  visual_cloze = [item for item in data if item["task"]=="visual_cloze"]
  visual_coherence = [item for item in data if item["task"]=="visual_coherence"]
  visual_ordering = [item for item in data if item["task"]=="visual_ordering"]
  return textual_cloze, visual_cloze, visual_coherence, visual_ordering

In [16]:
def delete_keys(dataset):
  match = re.compile('[^A-Za-z]')
  s = ""
  if not dataset[0]["task"]=="textual_cloze":
    for data in dataset:
      for step in data['context']:
        step['title'] = re.sub(match,' ', step['title']).lower()
        step['body'] = re.sub(match,' ', step['body']).lower()
        s += step['title']+ step['body']
  else:
    for data in dataset:
      for step in data['context']:
        step['body'] = re.sub(match,' ', step['body']).lower()
        s += step['body']
      for i in range(len(data['choice_list'])):
        data['choice_list'][i] = re.sub(match,' ', data['choice_list'][i]).lower()
        s += data['choice_list'][i]
      for i in range(len(data['question'])):
        data['question'][i] = re.sub(match,' ', data['question'][i]).lower()
        s += data['question'][i]

  if (dataset[0]["task"]=="visual_coherence"):
    [data.pop("question") for data in dataset]
  if (dataset[0]["task"]=="visual_ordering"):
    [data.pop("question") for data in dataset]
    for data in dataset:
      order = data["choice_list"][0]
      data["image_list"] = {0:order[0], 1:order[1], 2:order[2], 3:order[3]}
      img2ind = {v:k for k,v in data["image_list"].items()}
      data["choice_list"]=[[img2ind[img] for img in choice] for choice in data["choice_list"]]
  [[data.pop(key,None) for key in ["context_modality", "split", "qid", "question_modality", "task", "question_text"]] for data in dataset]
  [[[step.pop(key) for key in ["id","videos"]] for step in data['context']] for data in dataset]

  return set(s.split())
#  print(json.dumps(dataset[0], indent=4))

In [17]:
def load_image(file_path):
# Load image
  image = tf.io.decode_jpeg(tf.io.read_file(file_path),channels=3)
# Convert image to normalized float [0, 1]
  image = tf.image.convert_image_dtype(image,tf.float32)
# resize image
  image = tf.image.resize(image, [256,256])
# Rescale data to range (-1, 1)
  image = (image - 0.5) * 2
  return image

In [18]:
from copy import deepcopy
def data_iter(batch_size, dataset, task, split):
  num_input = len(dataset)
  np.random.shuffle(dataset)
  for i in range(num_input // batch_size):
    Xs = deepcopy(dataset[i*batch_size:(i+1)*batch_size])
    Ys = [item.pop("answer") for item in Xs]
    if task=="textual_cloze":
      for X in Xs:
        for step in X["context"]:
          step["images"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) for item in step["images"]]
    if task=="visual_cloze":
      for X in Xs:
        X["choice_list"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) for item in X["choice_list"]]
        X["question"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) if not item=="@placeholder" else "@placeholder"  for item in X["question"]]
    if task=="visual_coherence":
      for X in Xs:
        X["choice_list"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) for item in X["choice_list"]]
    if task=="visual_ordering":
      for X in Xs:
        for k,v in X["image_list"].items():
          X["image_list"][k] = load_image("data/images/images-qa/"+split+"/images-qa/"+X["image_list"][k])
    yield Xs, Ys



In [19]:
def get_vocab(split):
  textual_cloze, visual_cloze, visual_coherence, visual_ordering=read_file("data/"+split+" recipeqa.json")
  v1 = delete_keys(textual_cloze)
  v2 = delete_keys(visual_cloze)
  v3 = delete_keys(visual_coherence)
  v4 = delete_keys(visual_ordering)
  return textual_cloze, visual_cloze, visual_coherence, visual_ordering, v1.union(v2,v3,v4)


In [20]:
def load_embeddings(path_to_glove_file = 'glove.6B.100d.txt'):
  embedding_index = {}
  with open(path_to_glove_file) as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, "f", sep=" ")
      embedding_index[word] = coefs
  return embedding_index


In [21]:
def tokenize(textual_cloze, visual_cloze, visual_coherence, visual_ordering, word_index):
  for dataset in (visual_cloze, visual_coherence, visual_ordering):
    for data in dataset:
      for step in data['context']:
        step['title'] = [word_index[item] if item in word_index.keys() else 0 for item in step['title'].split()]
        step['body'] = [word_index[item] if item in word_index.keys() else 0 for item in step['body'].split()]
  for data in textual_cloze:
    for step in data['context']:
      step['body'] = [word_index[item] if item in word_index.keys() else 0 for item in step['body'].split()]
    for i in range(len(data['choice_list'])):
      data['choice_list'][i] = [word_index[item] if item in word_index.keys() else 0 for item in data['choice_list'][i].split()]
    for i in range(len(data['question'])):
      if not data['question'][i]==" placeholder":
        data['question'][i] = [word_index[item] if item in word_index.keys() else 0 for item in data['question'][i].split()]
      else:
        data['question'][i] = "@placeholder"

In [22]:
def get_iterator(textual_cloze, visual_cloze, visual_coherence, visual_ordering, batch_size, split):
  textual_cloze_iter = data_iter(batch_size, textual_cloze, "textual_cloze", split)
  visual_cloze_iter = data_iter(batch_size, visual_cloze, "visual_cloze", split)
  visual_coherence_iter = data_iter(batch_size, visual_coherence, "visual_coherence", split)
  visual_ordering_iter = data_iter(batch_size, visual_ordering, "visual_ordering", split)
  return textual_cloze_iter, visual_cloze_iter, visual_coherence_iter, visual_ordering_iter

In [23]:
def preprocess(batch_size=50):
  train1, train2, train3, train4, vtrain = get_vocab( "train")
  test1, test2, test3, test4,vtest = get_vocab("test")
  val1, val2, val3, val4, vval = get_vocab("val")
  vocab = list(vtrain.union(vtest, vval))
  embedding_index = load_embeddings()
  vocab = [item for item in vocab if item in embedding_index.keys()]
  vocab += ["*PAD*", "*UNK*"]
  vocab.reverse()
  word_index = {w:i for i,w in enumerate(vocab)}
  tokenize(train1, train2, train3, train4, word_index)
  tokenize(test1, test2, test3, test4, word_index)
  tokenize(val1, val2, val3, val4, word_index)
  train_iter1, train_iter2, train_iter3, train_iter4 = get_iterator(train1, train2, train3, train4, batch_size, "train")
  test_iter1, test_iter2, test_iter3, test_iter4 = get_iterator(test1, test2, test3, test4, batch_size, "test")
  val_iter1, val_iter2, val_iter3, val_iter4 = get_iterator(val1, val2, val3, val4, batch_size, "val")
  return (train_iter1, train_iter2, train_iter3, train_iter4), (test_iter1, test_iter2, test_iter3, test_iter4), (val_iter1, val_iter2, val_iter3, val_iter4), embedding_index, word_index

In [24]:
def get_embedding_layer(word_index, embeddings_index, embedding_dim = 100):
  num_tokens = len(word_index)
  embedding_matrix = np.zeros((num_tokens, embedding_dim))
  for word, i in word_index.items():
    if i<=1:
      continue
    embedding_matrix[i] = embeddings_index[word]
  embedding_layer = tf.keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
  )
  return embedding_layer

In [27]:
def main():
  batch_size = 50
  (train_iter1, train_iter2, train_iter3, train_iter4), (test_iter1, test_iter2, test_iter3, test_iter4), (val_iter1, val_iter2, val_iter3, val_iter4), embedding_index, word_index = preprocess(batch_size)
  # Usage:
  for Xs, Ys in val_iter1:
    print(json.dumps(Xs[0], indent=4, default=lambda x:"tf_tensor"))
    print(Ys[0])
    break
  print(len(word_index))
  print(list(word_index.keys())[:10])
  l_embed = get_embedding_layer(word_index, embedding_index)
  l_embed(tf.convert_to_tensor([[2]]))
  print(l_embed.get_weights()[0])

In [28]:
if __name__ == '__main__':
  main()


{
    "recipe_id": "donkey-kong-coconut-barrels-jello-shots",
    "context": [
        {
            "body": [
                19963,
                18261,
                25127,
                27344,
                25127,
                3801,
                3413,
                3318,
                25127,
                3528,
                11018,
                5475,
                21035,
                6004,
                25127,
                27048,
                12140,
                2256,
                1774,
                26113,
                6004,
                16503,
                26605,
                26980,
                3413,
                3201,
                25127,
                15206,
                15653,
                0,
                20026,
                25127,
                4113,
                25127,
                23904,
                20026,
                25127,
                4113,
                16503,
         